In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json

In [2]:
dirs4 = [(0, 1), (0, -1), (1, 0), (-1, 0)]
dirs8 = dirs4 + [(1, 1), (1, -1), (-1, 1), (-1, -1)]

In [3]:
boss = {}

In [4]:
def play(boss_hp, player_hp, mana, shield, poison, recharge, max_mana_left):
    if player_hp <= 0:
        return False

    poison_dmg = 3*min(2, poison)
    new_mana = mana+101*min(2, recharge)

    # Magic Missile
    mana_cost = 53
    dmg = 4+poison_dmg
    boss_dmg = max(0, boss['dmg']-(7 if shield >= 2 else 0))
    if mana_cost <= max_mana_left and mana_cost <= mana+(101 if recharge else 0):
        if boss_hp <= dmg:
            return True
        if play(boss_hp-dmg,
                player_hp-boss_dmg,
                new_mana-mana_cost,
                max(0, shield-2),
                max(0, poison-2),
                max(0, recharge-2),
                max_mana_left-mana_cost):
            return True

    # Drain
    mana_cost = 73
    dmg = 2+poison_dmg
    boss_dmg = max(0, boss['dmg']-(7 if shield >= 2 else 0))
    if mana_cost <= max_mana_left and mana_cost <= mana+(101 if recharge else 0):
        if boss_hp <= dmg:
            return True
        if play(boss_hp-dmg,
                player_hp+2-boss_dmg,
                new_mana-mana_cost,
                max(0, shield-2),
                max(0, poison-2),
                max(0, recharge-2),
                max_mana_left-mana_cost):
            return True

    # Shield
    mana_cost = 113
    dmg = poison_dmg
    boss_dmg = max(0, boss['dmg']-7)
    if shield <= 1 and mana_cost <= max_mana_left and mana_cost <= mana+(101 if recharge else 0):
        if boss_hp <= dmg:
            return True
        if play(boss_hp-dmg,
                player_hp-boss_dmg,
                new_mana-mana_cost,
                5,
                max(0, poison-2),
                max(0, recharge-2),
                max_mana_left-mana_cost):
            return True

    # Poison
    mana_cost = 173
    dmg = 3+poison_dmg
    boss_dmg = max(0, boss['dmg']-(7 if shield >= 2 else 0))
    if poison <= 1 and mana_cost <= max_mana_left and mana_cost <= mana+(101 if recharge else 0):
        if boss_hp <= dmg:
            return True
        if play(boss_hp-dmg,
                player_hp-boss_dmg,
                new_mana-mana_cost,
                max(0, shield-2),
                5,
                max(0, recharge-2),
                max_mana_left-mana_cost):
            return True

    # Recharge
    mana_cost = 229
    dmg = poison_dmg
    boss_dmg = max(0, boss['dmg']-(7 if shield >= 2 else 0))
    if recharge <= 1 and mana_cost <= max_mana_left and mana_cost <= mana+(101 if recharge else 0):
        if boss_hp <= dmg:
            return True
        if play(boss_hp-dmg,
                player_hp-boss_dmg,
                new_mana+101-mana_cost,
                max(0, shield-2),
                max(0, poison-2),
                4,
                max_mana_left-mana_cost):
            return True

    return False

In [5]:
boss['dmg'] = 8
print(play(13, 10, 250, 0, 0, 0, 1000))

boss['dmg'] = 9
print(play(14, 10, 250, 0, 0, 0, 1000))

True
False


In [6]:
with open("./data/day22.txt") as f:
    data = f.read().rstrip().split('\n')
    boss['hp'] = int(data[0].split(' ')[-1])
    boss['dmg'] = int(data[1].split(' ')[-1])

In [7]:
play(boss['hp'], 50, 500, 0, 0, 0, 1000)

True

In [8]:
bisect_left(range(1000), 1, key=lambda max_mana: play(boss['hp'], 50, 500, 0, 0, 0, max_mana))

953

In [9]:
print(play(boss['hp'], 50, 500, 0, 0, 0, 952))
print(play(boss['hp'], 50, 500, 0, 0, 0, 953))

False
True


In [10]:
def play2(boss_hp, player_hp, mana, shield, poison, recharge, max_mana_left):
    player_hp -= 1
    if player_hp <= 0:
        return False

    poison_dmg = 3*min(2, poison)
    new_mana = mana+101*min(2, recharge)

    # Magic Missile
    mana_cost = 53
    dmg = 4+poison_dmg
    boss_dmg = max(0, boss['dmg']-(7 if shield >= 2 else 0))
    if mana_cost <= max_mana_left and mana_cost <= mana+(101 if recharge else 0):
        if boss_hp <= dmg:
            return True
        if play2(boss_hp-dmg,
                player_hp-boss_dmg,
                new_mana-mana_cost,
                max(0, shield-2),
                max(0, poison-2),
                max(0, recharge-2),
                max_mana_left-mana_cost):
            return True

    # Drain
    mana_cost = 73
    dmg = 2+poison_dmg
    boss_dmg = max(0, boss['dmg']-(7 if shield >= 2 else 0))
    if mana_cost <= max_mana_left and mana_cost <= mana+(101 if recharge else 0):
        if boss_hp <= dmg:
            return True
        if play2(boss_hp-dmg,
                player_hp+2-boss_dmg,
                new_mana-mana_cost,
                max(0, shield-2),
                max(0, poison-2),
                max(0, recharge-2),
                max_mana_left-mana_cost):
            return True

    # Shield
    mana_cost = 113
    dmg = poison_dmg
    boss_dmg = max(0, boss['dmg']-7)
    if shield <= 1 and mana_cost <= max_mana_left and mana_cost <= mana+(101 if recharge else 0):
        if boss_hp <= dmg:
            return True
        if play2(boss_hp-dmg,
                player_hp-boss_dmg,
                new_mana-mana_cost,
                5,
                max(0, poison-2),
                max(0, recharge-2),
                max_mana_left-mana_cost):
            return True

    # Poison
    mana_cost = 173
    dmg = 3+poison_dmg
    boss_dmg = max(0, boss['dmg']-(7 if shield >= 2 else 0))
    if poison <= 1 and mana_cost <= max_mana_left and mana_cost <= mana+(101 if recharge else 0):
        if boss_hp <= dmg:
            return True
        if play2(boss_hp-dmg,
                player_hp-boss_dmg,
                new_mana-mana_cost,
                max(0, shield-2),
                5,
                max(0, recharge-2),
                max_mana_left-mana_cost):
            return True

    # Recharge
    mana_cost = 229
    dmg = poison_dmg
    boss_dmg = max(0, boss['dmg']-(7 if shield >= 2 else 0))
    if recharge <= 1 and mana_cost <= max_mana_left and mana_cost <= mana+(101 if recharge else 0):
        if boss_hp <= dmg:
            return True
        if play2(boss_hp-dmg,
                player_hp-boss_dmg,
                new_mana+101-mana_cost,
                max(0, shield-2),
                max(0, poison-2),
                4,
                max_mana_left-mana_cost):
            return True

    return False

In [11]:
print(play2(boss['hp'], 50, 500, 0, 0, 0, 1000))
print(play2(boss['hp'], 50, 500, 0, 0, 0, 1500))

False
True


In [12]:
bisect_left(range(1501), 1, lo=1000, key=lambda max_mana: play2(boss['hp'], 50, 500, 0, 0, 0, max_mana))

1289

In [13]:
print(play2(boss['hp'], 50, 500, 0, 0, 0, 1288))
print(play2(boss['hp'], 50, 500, 0, 0, 0, 1289))

False
True
